In [2]:
# 1. 사용환경 준비
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key 입력: ")

In [3]:
#2. 모델 준비
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini")


In [4]:
#3. 문서 로드하기
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력 / 인공지능 산업의 최신동향.pdf
loader = PyPDFLoader("C:/Users/82103/Desktop/과제파일/인공지능 산업의 최신동향.pdf")

# 페이지 별 문서 로드
docs = loader.load()

#### **청킹 방식**
1. 첫 번째 청킹 방식 : CharacterTextSplitter: 텍스트를 단순히 지정한 크기(chunk_size)로 자른다.  
2. 두 번째 청킹 방식 : RecursiveCharacterTextSplitter: 텍스트를 더 자연스럽게 나누기 위해 문장의 구조를 고려하며 점진적으로 쪼갠다.  


#### **청킹의 각 parameter의 뜻**

1. chunk_size : 한 청크의 최대 길이.
2. chunk_overlap : 청크 간의 겹치는 문자 수. 컨텍스트 연속성을 유지하기 위해 사용. (청크의 끝부분이 다음 청크의 시작부분과 겹치도록 한다.)
3. length_function : 청크의 길이를 측정.
4. separator : 텍스트를 분할하는 기준.
5. is_separator_regex : 말 그대로 separator를 정규식으로 사용할지 여부. True로 설정하면, 정규식으로 인식. False로 설정하면, 단순 문자열로 인식.


In [5]:
#4. 문서 청크로 나누기 / 청킹된 내용을 상위 10개까지 출력, 각 청킹방식과 parameter의 뜻을 markdown으로 정리.
#첫 번째 청킹 방식 : CharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

splits = text_splitter.split_documents(docs)

In [ ]:
#4. 문서 청크로 나누기 / 청킹된 내용을 상위 10개까지 출력, 각 청킹방식과 parameter의 뜻을 markdown으로 정리.
#두 번째 청킹 방식 : RecursiveCharacterTextSplitter 
from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

splits = recursive_text_splitter.split_documents(docs)

In [6]:
#5. 벡터 임베딩 생성
from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [7]:
#6. 벡터 스토어 생성
import faiss
from langchain_community.vectorstores import FAISS


vector_store = FAISS.from_documents(documents=splits, embedding=embeddings)

In [8]:
#7. FAISS를 Retriever로 변환
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 1})

In [9]:
#8. 프롬프트 템플릿 정의 
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

In [10]:
#9. RAG 체인 구성
from langchain.chains import LLMChain

class SimplePassThrough:
    def invoke(self, inputs, **kwargs):
        return inputs

class ContextToPrompt:
    def __init__(self, prompt_template):
        self.prompt_template = prompt_template
    
    def invoke(self, inputs):
        # 문서 내용을 텍스트로 변환
        if isinstance(inputs, list):
            context_text = "\n".join([doc.page_content for doc in inputs])
        else:
            context_text = inputs
        
        # 프롬프트 템플릿에 적용
        formatted_prompt = self.prompt_template.format_messages(
            context=context_text,
            question=inputs.get("question", "")
        )
        return formatted_prompt

# Retriever를 invoke() 메서드로 래핑하는 클래스 정의
class RetrieverWrapper:
    def __init__(self, retriever):
        self.retriever = retriever

    def invoke(self, inputs):
        if isinstance(inputs, dict):
            query = inputs.get("question", "")
        else:
            query = inputs
        # 검색 수행
        response_docs = self.retriever.get_relevant_documents(query)
        return response_docs

llm_chain = LLMChain(llm=model, prompt=contextual_prompt)

# RAG 체인 설정
rag_chain_debug = {
    "context": RetrieverWrapper(retriever),
    "prompt": ContextToPrompt(contextual_prompt),
    "llm": model
}

C:\Users\82103\AppData\Local\Temp\ipykernel_5872\1373480811.py:40: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  llm_chain = LLMChain(llm=model, prompt=contextual_prompt)


#### **RAG의 필요성**
1. 외부 데이터베이스나 검색 시스템으로 최신 정보와 특정 주제에 대한 답을 생성. 검색하고 참고하여 생성하기에, "__정확성과 신뢰도__" 증가.
2. 필요한 정보만 실시간 검색하여 사용. 따라서, 모델 크기를 줄이고 "__메모리 자원을 효율적__" 으로 활용 가능.
3. 데이터를 동적으로 연결하여, 언어 모델을 "__특정 도메인에 최적화__" 가능.

In [ ]:
#10. 챗봇 구동 확인 / 같은 질문을 일반 chat gpt와 답변을 비교해보고, 왜 RAG이 필요한지 간단히 markdown으로 서술.
# 챗봇 구동
while True:
    print("========================")
    query = input("질문을 입력하세요 : ")
    
    # 1. Retriever로 관련 문서 검색
    response_docs = rag_chain_debug["context"].invoke({"question": query})
    
    # 2. 문서를 프롬프트로 변환
    prompt_messages = rag_chain_debug["prompt"].invoke({
        "context": response_docs,
        "question": query
    })
    
    # 3. LLM으로 응답 생성
    response = rag_chain_debug["llm"].invoke(prompt_messages)
    
    print("\n답변:")
    print(response.content)

#### **질문**

"최신 인공지능의 동향을 알려줘"

#### **챗봇의 답변**

최신 인공지능 동향에 대한 구체적인 내용은 제공된 문서의 2페이지에 위치해 있습니다. 문서에서는 인공지능 산업 동향에 대한 브리프를 다루고 있습니다. 더 자세한 정보를 원하신다면 해당 문서를 참고하시기 바랍니다.

#### **ChatGPT의 답변**

#사이트 5개 검색함


2024년의 인공지능(AI)은 다양한 기술적 발전과 응용으로 큰 주목을 받고 있습니다. 주요 동향은 다음과 같습니다.

1. 생성형 AI와 다중 모달 시스템
생성형 AI는 기존 텍스트 중심에서 이미지, 오디오, 동영상 데이터를 포함하는 다중 모달 AI 시스템으로 진화하고 있습니다. 이 시스템은 인간처럼 다양한 감각 데이터를 통합해 문제를 해결할 수 있으며, 자동차의 피로 감지, 보험 클레임 관리 등 여러 산업에서 활용됩니다​
NVIDIA BLOG KOREA
​
고품질 학습데이터가 AI 모델의 성공을 만듭니다 | APPEN 에펜
.
2. AI와 과학의 융합
생명과학과 재료 과학 분야에서 AI는 혁신적인 연구 속도를 크게 높이고 있습니다. 예를 들어, 암 치료에 사용되는 대규모 AI 모델과 신약 개발을 위한 AI 기술이 수년 걸리던 과정을 수개월로 단축하고 있습니다​
인공지능신문
.
3. AI 윤리와 규제
AI 활용이 확산됨에 따라, 데이터 프라이버시, 투명성, 그리고 책임성을 강화하기 위한 윤리적 기준과 규제 프레임워크가 강화되고 있습니다. NIST AI 위험 관리 프레임워크와 같은 표준이 대규모 기업에서 AI 채택을 촉진하고 있습니다​
추천 : 네이버 블로그
​
고품질 학습데이터가 AI 모델의 성공을 만듭니다 | APPEN 에펜
.
4. AI의 상업적 활용 증가
기업들은 고객 맞춤형 응용 프로그램과 지능형 비즈니스 도구를 제공하기 위해 AI API를 통합하고 있습니다. 이와 함께, 기업 데이터에 최적화된 RAG(Retrieval-Augmented Generation) 기술이 부정확한 정보 생성을 줄이고 있습니다​
NVIDIA BLOG KOREA
​
추천 : 네이버 블로그
.
5. AI와 지속가능성
AI는 농업과 환경 보전에서도 두각을 나타냅니다. 예를 들어, 기후변화 완화를 위한 일기예보 시스템과 농업 생산성을 높이는 도구가 개발되고 있습니다​
인공지능신문
.
결론
2024년은 AI 기술의 진화가 산업과 일상생활에 깊숙이 스며들면서, 도전과 기회를 동시에 제공하는 해가 될 것으로 보입니다. 이를 기반으로 스마트 도시, 의료 혁신, 그리고 지속 가능한 환경을 구축할 가능성이 열리고 있습니다.

#### **챗봇과 ChatGPT의 답변 비교**

둘의 내용을 비교하여 봤을 때, 기본적으로 챗봇은 현재 제공된 문서에 기반하여 정보를 전달해 준다.

반면에 ChatGPT의 경우는, 여러 사이트를 검색한 모습을 명시적으로 확인할 수 있었다. 그래서 그에 따른 정보들도 신뢰와 정확도 측면에서 더 강화된 느낌을 받았다.
위에 짧게 언급한 부분을 쉽고 빠르게 체감하니 RAG의 필요성을 느끼기에 충분했다.